In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import tsfel.feature_extraction as tsf

import numpy as np

import torchaudio

In [2]:
X = np.random.randn(100, 26, 200)
Y = torch.Tensor(X)

In [11]:
def compute_time(signal, fs, dim):
    return torch.arange(0, signal.shape[dim]) / fs
print(compute_time(Y, 1, dim=1), tsf.compute_time(X[0,:,0], 1))

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.]) [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25.]


In [21]:
def calc_centroid(signal, fs, dim):
    time = compute_time(signal, fs, dim)
    energy = signal ** 2
    shape = [None] * len(signal.shape)
    shape[dim] = slice(None, None, None)
    t_energy = torch.sum(torch.multiply(time[shape], energy), dim)
    energy_sum = torch.sum(energy, dim)
    centroid = t_energy / energy_sum
    return centroid
print(calc_centroid(Y, 1, dim=1)[0,0], tsf.calc_centroid(X[0,:,0], 1))

tensor(13.5332) 13.53324715943185


In [34]:
def negative_turning(signal, dim):
    diff_sig = torch.diff(signal, dim=dim)
    shape1 = [slice(None, None, None)] * len(signal.shape)
    shape1[dim] = slice(None, -1, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = slice(1, None, None)
    diff_sig_sign = torch.sign(diff_sig)
    sign_diff = diff_sig_sign[shape1] * diff_sig_sign[shape2]
    return torch.sum((sign_diff<0) & (diff_sig_sign[shape1]>0), dim=dim)

print(negative_turning(Y, dim=1)[0,0], tsf.negative_turning(X[0,:,0]))

tensor(7) 8


In [35]:
def positive_turning(signal, dim):
    diff_sig = torch.diff(signal, dim=dim)
    shape1 = [slice(None, None, None)] * len(signal.shape)
    shape1[dim] = slice(None, -1, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = slice(1, None, None)
    diff_sig_sign = torch.sign(diff_sig)
    sign_diff = diff_sig_sign[shape1] * diff_sig_sign[shape2]
    return torch.sum((sign_diff<0) & (diff_sig_sign[shape1]<0), dim=dim)

print(positive_turning(Y, dim=1)[0,0], tsf.positive_turning(X[0,:,0]))

tensor(8) 7


In [38]:
def mean_abs_diff(signal, dim):
    return torch.mean(torch.abs(torch.diff(signal, dim=dim)), dim=dim)

print(mean_abs_diff(Y, dim=1)[0,0], tsf.mean_abs_diff(X[0,:,0]))

tensor(1.0180) 1.0180389070820206


In [40]:
def mean_diff(signal, dim):
    return torch.mean(torch.diff(signal, dim=dim), dim=dim)
print(mean_diff(Y, dim=1)[0,0], tsf.mean_diff(X[0,:,0]))

tensor(0.0241) 0.024078354745554448


In [42]:
def median_abs_diff(signal, dim):
    return torch.median(torch.abs(torch.diff(signal, dim=dim)), dim=dim)[0]
print(median_abs_diff(Y, dim=1)[0,0], tsf.median_abs_diff(X[0,:,0]))

tensor(0.9164) 0.9164040850137399


In [43]:
def median_diff(signal, dim):
    return torch.median(torch.diff(signal, dim=dim), dim=dim)[0]
print(median_diff(Y, dim=1)[0,0], tsf.median_diff(X[0,:,0]))

tensor(-0.1051) -0.1050789752705468


In [47]:
def distance(signal, dim):
    diff_sig = torch.diff(signal, dim=dim)
    return torch.sum(torch.sqrt(1 + diff_sig**2), dim)
print(distance(Y, dim=1)[0,0], tsf.distance(X[0,:,0]))

tensor(36.6801) 36.6801232043038


In [49]:
def sum_abs_diff(signal, dim):
    return torch.sum(torch.abs(torch.diff(signal, dim=dim)), dim=dim)
print(sum_abs_diff(Y, dim=1)[0,0], tsf.sum_abs_diff(X[0,:,0]))

tensor(25.4510) 25.450972677050515


In [51]:
def zero_cross(signal, dim):
    return torch.sum(torch.diff(torch.sign(signal), dim=dim).bool(), dim=dim)
print(zero_cross(Y, dim=1)[0,0], tsf.zero_cross(X[0,:,0]))

tensor(14) 14


In [55]:
def auc(signal, fs, dim):
    t = compute_time(signal, fs, dim)
    shape = [None] * len(signal.shape)
    shape[dim] = slice(None, None, None)
    shape1 = [slice(None, None, None)] * len(signal.shape)
    shape1[dim] = slice(None, -1, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = slice(1, None, None)
    return torch.sum(0.5 * torch.diff(t)[shape] * torch.abs(signal[shape1] + signal[shape2]), dim=dim)
print(auc(Y, 1, dim=1)[0,0], tsf.auc(X[0,:,0], 1))

tensor(12.5816) 12.58162228287284


In [57]:
def abs_energy(signal, dim):
    return torch.sum(torch.abs(signal) ** 2, dim=dim)
print(abs_energy(Y, dim=1)[0,0], tsf.abs_energy(X[0,:,0]))

tensor(18.3739) 18.37385219909701


In [59]:
def average_power(signal, fs, dim):
    time = compute_time(signal, fs, dim)
    return torch.sum(signal ** 2, dim=dim) / (time[-1] - time[0])
print(average_power(Y, 1, dim=1)[0,0], tsf.average_power(X[0,:,0], 1))

tensor(0.7350) 0.7349540879638803


In [86]:
# cross entropy
def create_xx(features):
    if max(features) < 0:
        max_f = -torch.max(features,dim=0)[0]
        min_f = torch.min(features,dim=0)[0]
    else:
        min_f = torch.min(features,dim=0)[0]
        max_f = torch.max(features,dim=0)[0]
    if torch.min(features,dim=0)[0] == torch.max(features,dim=0)[0]:
        xx = torch.linspace(min_f, min_f + 10, len(features))
    else:
        xx = torch.linspace(min_f, max_f, len(features))
    return xx

def gaussian(features):
    xx = create_xx(features)
    std_value = torch.std(features)
    mean_value = torch.mean(features)
    if std_value == 0:
        return 0.0
    pdf_gauss = torch.distributions.normal.Normal(mean_value, std_value).log_prob(xx).exp()
    return pdf_gauss / torch.sum(pdf_gauss)

def entropy(signal, prob="gauss"):
    if prob == "standard":
        value, counts = torch.unique(signal, return_counts=True)
        p = counts / counts.sum()
    elif prob == "gauss":
        p = gaussian(signal)
    if torch.sum(p) == 0:
        return 0.0
    # Handling zero probability values
    p = p[torch.where(p != 0)]
    # If probability all in one value, there is no entropy
    if np.log2(len(signal)) == 1:
        return 0.0
    elif torch.sum(p * np.log2(p)) / np.log2(len(signal)) == 0:
        return 0.0
    else:
        return -torch.sum(p * np.log2(p)) / np.log2(len(signal))
print(entropy(Y), tsf.entropy(X))
np.isclose(entropy(Y), tsf.entropy(X)).all()

tensor(0.9429) 1.0


/tmp/ipykernel_8324/3421186131.py:37: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  elif torch.sum(p * np.log2(p)) / np.log2(len(signal)) == 0:
/tmp/ipykernel_8324/3421186131.py:40: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  return -torch.sum(p * np.log2(p)) / np.log2(len(signal))


np.False_

In [51]:
# histogram along a dim ???
def hist_mode(signal, nbins=10):
    hist_values, bin_edges = torch.histogram(signal, bins=nbins)
    max_bin_idx = torch.argmax(hist_values)
    mode_value = (bin_edges[max_bin_idx] + bin_edges[max_bin_idx + 1]) / 2.0
    return mode_value
print(hist_mode(Y, dim=1)[0,0], tsf.hist_mode(X[0,:,0]))

-0.25253606 -0.25253615296528875


np.True_

In [63]:
def interq_range(signal, dim):
    return torch.quantile(signal, .75, dim=dim) - torch.quantile(signal, .25, dim=dim)
print(interq_range(Y, dim=1)[0,0], tsf.interq_range(X[0,:,0]))

tensor(1.1355) 1.1355037340897236


In [64]:
def pk_pk_distance(signal, dim):
    return torch.abs(torch.max(signal, dim=dim)[0] - torch.min(signal, dim=dim)[0])
print(pk_pk_distance(Y, dim=1)[0,0], tsf.pk_pk_distance(X[0,:,0]))

tensor(3.2980) 3.2980223870880443


In [68]:
def calc_ecdf(signal, dim):
    return torch.sort(signal, dim=dim)[0], torch.arange(1, signal.shape[dim] + 1) / signal.shape[dim]

In [83]:
def ecdf_percentile(signal, dim, percentile):
    x, y = calc_ecdf(signal, dim)
    shape = [None] * len(signal.shape)
    shape[dim] = slice(None, None, None)
    mask = y <= percentile
    mask = mask[shape]
    mask = mask.expand(signal.shape)
    return torch.amax(torch.masked.as_masked_tensor(x, mask), dim=dim)
print(ecdf_percentile(Y, dim=1, percentile=0.75).get_data()[0,0], tsf.ecdf_percentile(X[0,:,0], percentile=0.75))

tensor(0.7358) 0.7357521728830281


In [84]:
def ecdf_slope(signal, dim, p_init=0.5, p_end=0.75):
    x_init = ecdf_percentile(signal, dim = dim, percentile=p_init).get_data()
    x_end = ecdf_percentile(signal, dim = dim, percentile=p_end).get_data()
    return (p_end - p_init) / (x_end - x_init)
print(ecdf_slope(Y, dim=1)[0,0], tsf.ecdf_slope(X[0,:,0]))

tensor(0.3939) 0.3938736973107223


In [85]:
def calc_fft(signal, fs, dim):
    fmag = torch.abs(torch.fft.rfft(signal, dim=dim))
    f = torch.fft.rfftfreq(signal.shape[dim], d=1 / fs)
    return f, fmag

In [ ]:
# nor related to signal itself?
def spectral_distance(signal, fs, dim):
    f, fmag = calc_fft(signal, fs)
    cum_fmag = torch.cumsum(fmag, dim=dim)
    # Computing the linear regression
    points_y = torch.linspace(0, cum_fmag[-1], len(cum_fmag))
    return torch.sum(points_y - cum_fmag)
print(spectral_distance(Y, 1, dim=1)[0,0], tsf.spectral_distance(X[0,:,0], 1))

In [88]:
def max_power_spectrum(signal, fs):
    if np.std(signal) == 0:
        return float(max(scipy.signal.welch(signal, fs, nperseg=len(signal))[1]))
    else:
        return float(max(scipy.signal.welch(signal / np.std(signal), fs, nperseg=len(signal))[1]))

In [94]:
def spectral_centroid(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    shape = [None] * len(signal.shape)
    shape[dim] = slice(None, None, None)
    return torch.sum(torch.multiply(f[shape], fmag / torch.sum(fmag, keepdim=True, dim=dim)), dim=dim)
print(spectral_centroid(Y, 1, dim=1)[0,0], tsf.spectral_centroid(X[0,:,0], 1))

tensor(0.2294) 0.2294147586963452


In [102]:
def spectral_spread(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    spect_centroid = spectral_centroid(signal, fs, dim)
    shape1 = [None] * len(signal.shape)
    shape1[dim] = slice(None, None, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = None
    return torch.sum(torch.multiply(((f[shape1] - spect_centroid[shape2]) ** 2), (fmag / torch.sum(fmag, keepdim=True, dim=dim))), dim=dim) ** 0.5
print(spectral_spread(Y, 1, dim=1)[0,0], tsf.spectral_spread(X[0,:,0], 1))

tensor(0.1481) 0.14810541841586658


In [118]:
def max_frequency(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    cum_fmag = torch.cumsum(fmag, dim=dim)
    shape = [slice(None, None, None)] * len(signal.shape)
    shape[dim] = slice(-1, None, None)
    try:
        ind_mag = torch.sum(~(cum_fmag > cum_fmag[shape] * 0.95), dim)
    except IndexError:
        ind_mag = torch.argmax(cum_fmag, dim=dim)
    print(ind_mag[0,0])
    return f[ind_mag]
print(max_frequency(Y, 1, dim=1)[0,0], tsf.max_frequency(X[0,:,0], 1))

tensor(13)
tensor(0.5000) 0.5


In [132]:
def median_frequency(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    cum_fmag = torch.cumsum(fmag, dim=dim)
    shape = [slice(None, None, None)] * len(signal.shape)
    shape[dim] = slice(-1, None, None)
    try:
        ind_mag = torch.sum(~(cum_fmag > cum_fmag[shape] * 0.5), dim)
    except IndexError:
        ind_mag = torch.argmax(cum_fmag, dim=dim)
    return f[ind_mag]
print(median_frequency(Y, 1, dim=1)[0,0], tsf.median_frequency(X[0,:,0], 1))

tensor(0.2308) 0.23076923076923078


In [130]:
def spectral_decrease(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    shape = [slice(None, None, None)] * len(signal.shape)
    shape[dim] = slice(1, None, None)
    fmag_band = fmag[shape]
    len_fmag_band = torch.arange(2, fmag.shape[dim] + 1)
    shape1 = [slice(None, None, None)] * len(signal.shape)
    shape1[dim] = slice(0, 1, None)
    shape2 = [None] * len(signal.shape)
    shape2[dim] = slice(None, None, None)
    soma_num = torch.sum((fmag_band - fmag[shape1]) / (len_fmag_band - 1)[shape2], dim=dim)
    soma_den = 1 / torch.sum(fmag_band, dim=dim)
    return soma_den * soma_num
print(spectral_decrease(Y, 1, dim=1)[0,0], tsf.spectral_decrease(X[0,:,0], 1))

tensor(-0.1645) -0.1645408797869453


In [136]:
def spectral_kurtosis(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    shape1 = [None] * len(signal.shape)
    shape1[dim] = slice(None, None, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = None
    spect_kurt = ((f[shape1] - spectral_centroid(signal, fs, dim)[shape2]) ** 4) * (fmag / torch.sum(fmag, keepdim=True, dim=dim))
    return torch.sum(spect_kurt, dim=dim) / (spectral_spread(signal, fs, dim) ** 4)
print(spectral_kurtosis(Y, 1, dim=1)[0,0], tsf.spectral_kurtosis(X[0,:,0], 1))

tensor(1.9984) 1.9983559325528717


In [ ]:
def max_frequency(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    cum_fmag = torch.cumsum(fmag, dim=dim)
    shape = [slice(None, None, None)] * len(signal.shape)
    shape[dim] = slice(-1, None, None)
    try:
        ind_mag = torch.sum(~(cum_fmag > cum_fmag[shape] * 0.95), dim)
    except IndexError:
        ind_mag = torch.argmax(cum_fmag, dim=dim)
    print(ind_mag[0,0])
    return f[ind_mag]
print(max_frequency(Y, 1, dim=1)[0,0], tsf.max_frequency(X[0,:,0], 1))

tensor(13)
tensor(0.5000) 0.5


In [138]:
def spectral_skewness(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    shape1 = [None] * len(signal.shape)
    shape1[dim] = slice(None, None, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = None
    spect_kurt = ((f[shape1] - spectral_centroid(signal, fs, dim)[shape2]) ** 3) * (fmag / torch.sum(fmag, keepdim=True, dim=dim))
    return torch.sum(spect_kurt, dim=dim) / (spectral_spread(signal, fs, dim) ** 3)
print(spectral_skewness(Y, 1, dim=1)[0,0], tsf.spectral_skewness(X[0,:,0], 1))

tensor(0.1338) 0.1338421498049864


In [141]:
def spectral_slope(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    sum_fmag = torch.sum(fmag, dim=dim)
    dot_ff = (f * f).sum()
    sum_f = f.sum()
    len_f = len(f)
    shape = [None] * len(signal.shape)
    shape[dim] = slice(None, None, None)
    num_ = (1 / sum_fmag) * (len_f * torch.sum(f[shape] * fmag, dim=dim) - sum_f * sum_fmag)
    denom_ = len_f * dot_ff - sum_f**2
    return num_ / denom_
print(spectral_slope(Y, 1, dim=1)[0,0], tsf.spectral_slope(X[0,:,0], 1))

tensor(-0.0612) -0.0611675741594314


In [142]:
def spectral_variation(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    shape1 = [slice(None, None, None)] * len(signal.shape)
    shape1[dim] = slice(None, -1, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = slice(1, None, None)
    sum1 = torch.sum(fmag[shape1] * fmag[shape2], dim=dim)
    sum2 = torch.sum(fmag[shape2] ** 2, dim=dim)
    sum3 = torch.sum(fmag[shape1] ** 2, dim=dim)
    variation = 1 - (sum1 / ((sum2**0.5) * (sum3**0.5)))
    return variation
print(spectral_variation(Y, 1, dim=1)[0,0], tsf.spectral_variation(X[0,:,0], 1))

tensor(0.3954) 0.39538766507909007


In [143]:
def spectral_positive_turning(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    diff_sig = torch.diff(fmag, dim=dim)
    shape1 = [slice(None, None, None)] * len(signal.shape)
    shape1[dim] = slice(None, -1, None)
    shape2 = [slice(None, None, None)] * len(signal.shape)
    shape2[dim] = slice(1, None, None)
    diff_sig_sign = torch.sign(diff_sig)
    sign_diff = diff_sig_sign[shape1] * diff_sig_sign[shape2]
    return torch.sum((sign_diff<0) & (diff_sig_sign[shape1]<0), dim=dim)
print(spectral_positive_turning(Y, 1, dim=1)[0,0], tsf.spectral_positive_turning(X[0,:,0], 1))

tensor(6) 5


In [127]:
def spectral_roll_off(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    cum_ff = torch.cumsum(fmag, dim=dim)
    value = 0.95 * (torch.sum(fmag, keepdim=True, dim=dim))
    return f[torch.sum(~(cum_ff >= value), dim=dim)]
print(spectral_roll_off(Y, 1, dim=1)[0,0], tsf.spectral_roll_off(X[0,:,0], 1))

tensor(0.5000) 0.5


In [126]:
def spectral_roll_on(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    cum_ff = torch.cumsum(fmag, dim=dim)
    value = 0.05 * (torch.sum(fmag, keepdim=True, dim=dim))
    return f[torch.sum(~(cum_ff >= value), dim=dim)]
print(spectral_roll_on(Y, 1, dim=1)[0,0], tsf.spectral_roll_on(X[0,:,0], 1))

tensor(0.) 0.0


In [121]:
def human_range_energy(signal, fs, dim):
    f, fmag = calc_fft(signal, fs, dim)
    allenergy = torch.sum(fmag**2, dim=dim)
    shape = [slice(None, None, None)] * len(signal.shape)
    shape[dim] = slice(torch.argmin(torch.abs(0.6 - f)), torch.argmin(torch.abs(2.5 - f)), None)
    hr_energy = torch.sum(fmag[shape] ** 2, dim=dim)
    ratio = hr_energy / allenergy
    return ratio
print(human_range_energy(Y, 1, dim=1)[0,0], tsf.human_range_energy(X[0,:,0], 1))

tensor(0.) 0.0


In [ ]:
def filterbank(signal, fs, pre_emphasis=0.97, nfft=512, nfilt=40):
    """Computes the MEL-spaced filterbank.

    It provides the information about the power in each frequency band.

    Implementation details and description on:
    https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial
    https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html#fnref:1

    Parameters
    ----------
    signal : nd-array
        Input from which filterbank is computed
    fs : float
        Sampling frequency
    pre_emphasis : float
        Pre-emphasis coefficient for pre-emphasis filter application
    nfft : int
        Number of points of fft
    nfilt : int
        Number of filters

    Returns
    -------
    nd-array
        MEL-spaced filterbank
    """

    # Signal is already a window from the original signal, so no frame is needed.
    # According to the references it is needed the application of a window function such as
    # hann window. However if the signal windows don't have overlap, we will lose information,
    # as the application of a hann window will overshadow the windows signal edges.

    # pre-emphasis filter to amplify the high frequencies

    emphasized_signal = np.append(
        np.array(signal)[0],
        np.array(signal[1:]) - pre_emphasis * np.array(signal[:-1]),
    )

    # Fourier transform and Power spectrum
    mag_frames = np.absolute(
        np.fft.rfft(emphasized_signal, nfft),
    )  # Magnitude of the FFT

    pow_frames = (1.0 / nfft) * (mag_frames**2)  # Power Spectrum

    low_freq_mel = 0
    high_freq_mel = 2595 * np.log10(1 + (fs / 2) / 700)  # Convert Hz to Mel
    mel_points = np.linspace(
        low_freq_mel,
        high_freq_mel,
        nfilt + 2,
    )  # Equally spaced in Mel scale
    hz_points = 700 * (10 ** (mel_points / 2595) - 1)  # Convert Mel to Hz
    filter_bin = np.floor((nfft + 1) * hz_points / fs)

    fbank = np.zeros((nfilt, int(np.floor(nfft / 2 + 1))))
    for m in np.arange(1, nfilt + 1):

        f_m_minus = int(filter_bin[m - 1])  # left
        f_m = int(filter_bin[m])  # center
        f_m_plus = int(filter_bin[m + 1])  # right

        for k in np.arange(f_m_minus, f_m):
            fbank[m - 1, k] = (k - filter_bin[m - 1]) / (filter_bin[m] - filter_bin[m - 1])
        for k in np.arange(f_m, f_m_plus):
            fbank[m - 1, k] = (filter_bin[m + 1] - k) / (filter_bin[m + 1] - filter_bin[m])

    # Area Normalization
    # If we don't normalize the noise will increase with frequency because of the filter width.
    enorm = 2.0 / (hz_points[2 : nfilt + 2] - hz_points[:nfilt])
    fbank *= enorm[:, np.newaxis]

    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(
        filter_banks == 0,
        np.finfo(float).eps,
        filter_banks,
    )  # Numerical Stability
    filter_banks = 20 * np.log10(filter_banks)  # dB

    return filter_banks

def mfcc(signal, fs, pre_emphasis=0.97, nfft=512, nfilt=40, num_ceps=12, cep_lifter=22):
    filter_banks = filterbank(signal, fs, pre_emphasis, nfft, nfilt)
    mel_coeff = scipy.fft.dct(filter_banks, type=2, axis=0, norm="ortho")[1 : (num_ceps + 1)]  # Keep 2-13
    mel_coeff -= np.mean(mel_coeff, axis=0) + 1e-8
    # liftering
    ncoeff = len(mel_coeff)
    n = np.arange(ncoeff)
    lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)  # cep_lifter = 22 from python_speech_features library
    mel_coeff *= lift
    return tuple(mel_coeff)
print(mfcc(Y, 5), tsf.mfcc(X, 5))
np.isclose(mfcc(Y, 5), tsf.mfcc(X, 5)).all()

In [18]:
def _periodogram(X: torch.Tensor, fs, detrend, scaling):
    if X.dim() > 2:
        X = torch.squeeze(X)
    elif X.dim() == 1:
        X = X.unsqueeze(0)

    if detrend:
        X -= X.mean(-1, keepdim=True)

    N = X.size(-1)
    assert N % 2 == 0

    df = fs / N
    dt = df
    f = torch.arange(0, N / 2 + 1) * df  # [0:df:f/2]

    dual_side = torch.fft.fft(X)  # 双边谱
    half_idx = int(N / 2 + 1)
    single_side = dual_side[:, 0:half_idx]
    win = torch.abs(single_side)

    ps = win ** 2
    if scaling == 'density':  # 计算功率谱密度
        scale = N * fs
    elif scaling == 'spectrum':  # 计算功率谱
        scale = N ** 2
    elif scaling is None: # 不做缩放
        scale = 1
    else:
        raise ValueError('Unknown scaling: %r' % scaling)
    Pxy = ps / scale

    Pxy[:, 1:-1] *= 2  # 能量2倍;直流分量不用二倍, 中心频率点不用二倍

    return f, Pxy.squeeze()


def periodogram(X: torch.Tensor, fs=256, detrend=False, scaling='density', no_grad=True):
    """计算信号单边 PSD, 基本等价于 scipy.signal.periodogram
    
        Parameters:
        ----------
            - `X`:          torch.Tensor, EEG, [T]/[N, T]
            - `fs`:         int, 采样率, Hz
            - `detrend`:    bool, 是否去趋势 (去除直流分量)
            - `scaling`:    { 'density', 'spectrum' }, 可选
                - 'density':    计算功率谱密度 `(V ** 2 / Hz)`
                - 'spectrum':    计算功率谱 `(V ** 2)`
            - `no_grad`:    bool, 是否启用 no_grad() 模式
    
        Returns:
        ----------
            - `Pxy`:    Tensor, 单边功率谱
    """
    if no_grad:
        with torch.no_grad():
            return _periodogram(X, fs, detrend, scaling)
    else:
        return _periodogram(X, fs, detrend, scaling)
    
def _get_window(window, nwlen, device):
    if window == 'hann':
        window = torch.hann_window(
            nwlen, dtype=torch.float32, device=device, periodic=False
        )
    elif window == 'hamming':
        window = torch.hamming_window(
            nwlen, dtype=torch.float32, device=device, periodic=False
        )
    elif window == 'blackman':
        window = torch.blackman_window(
            nwlen, dtype=torch.float32, device=device, periodic=False
        )
    elif window == 'boxcar':
        window = torch.ones(nwlen, dtype=torch.float32, device=device)
    else:
        raise ValueError('Invalid Window {}' % window)
    return window


def _pwelch(X: torch.Tensor, fs, detrend, scaling, window, nwlen, nhop):
    if scaling == 'density':
        scale = (fs * (window * window).sum().item())
    elif scaling == 'spectrum':
        scale = window.sum().item() ** 2
    else:
        raise ValueError('Unknown scaling: %r' % scaling)
    # --------------- Fold and windowing --------------- #
    N, T = X.size(0), X.size(-1)
    X = X.view(N, 1, 1, T)
    X_fold = F.unfold(X, (1, nwlen), stride=nhop)  # [N, 1, 1, T] -> [N, nwlen, win_cnt]
    if detrend:
        X_fold -= X_fold.mean(1, keepdim=True) # 各个窗口各自detrend
    window = window.view(1, -1, 1)  # [1, nwlen, 1]
    X_windowed = X_fold * window  # [N, nwlen, win_cnt]
    win_cnt = X_windowed.size(-1)

    # --------------- Pwelch --------------- #
    X_windowed = X_windowed.transpose(1, 2).contiguous()  # [N, win_cnt, nwlen]
    X_windowed = X_windowed.view(N * win_cnt, nwlen)  # [N * win_cnt, nwlen]
    f, pxx = _periodogram(
        X_windowed, fs, detrend=False, scaling=None
    )  # [N * win_cnt, nwlen // 2 + 1]
    pxx /= scale
    pxx = pxx.view(N, win_cnt, -1)  # [N, win_cnt, nwlen // 2 + 1]
    pxx = torch.mean(pxx, dim=1)  # [N, nwlen // 2 + 1]
    return f, pxx


def pwelch(
    X: torch.Tensor,
    fs=256,
    detrend=False,
    scaling='density',
    window='hann',
    nwlen=128,
    nhop=None,
    no_grad=True,
):
    """Pwelch 方法，大致相当于 scipy.signal.welch

        Parameters:
        ----------
            - `X`:          torch.Tensor, EEG, [T]/[N, T]
            - `fs`:         int, 采样率, Hz
            - `detrend`:    bool, 是否去趋势 (去除直流分量)
            - `scaling`:    { 'density', 'spectrum' }, 可选
                - 'density':    计算功率谱密度 `(V ** 2 / Hz)`
                - 'spectrum':    计算功率谱 `(V ** 2)`
            - `window`:     str, 窗函数名称
            - `nwlen`:      int, 窗函数长度 (点的个数)
            - `nhop`:       int, 窗函数移动步长, 即 nwlen - noverlap (点的个数)
                            如果为 None，则默认为 `nwlen // 2`
            - `no_grad`:    bool, 是否启用 no_grad() 模式
    
        Returns:
        ----------
            - `Pxy`:    Tensor, 单边功率谱
    """
    nhop = nwlen // 2 if nhop is None else nhop
    window = _get_window(window, nwlen, X.device)
    if no_grad:
        with torch.no_grad():
            return _pwelch(X, fs, detrend, scaling, window, nwlen, nhop)
    else:
        return _pwelch(X, fs, detrend, scaling, window, nwlen, nhop)
pwelch(torch.stack([Y], dim=0), fs=1, window = "hann", scaling = 'density', detrend=True, nwlen=10, nhop=5)

(tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000]),
 tensor([[0.5287, 1.7273, 1.9764, 1.9345, 1.8635, 0.6566]]))

In [19]:
import scipy
scipy.signal.welch(X, fs=1., window = "hann", scaling = 'density', nperseg=10, noverlap=5)

(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5]),
 array([0.38725113, 1.77615456, 1.9841756 , 1.96212444, 1.90009807,
        0.66686779]))

In [142]:
def spectral_entropy(signal, fs):
    # Removing DC component
    sig = signal - torch.mean(signal)
    f, fmag = calc_fft(sig, fs)
    power = fmag**2
    if power.sum() == 0:
        return 0.0
    prob = torch.divide(power, power.sum())
    prob = prob[prob != 0]
    # If probability all in one value, there is no entropy
    if prob.size() == 1:
        return 0.0
    return -torch.multiply(prob, torch.log2(prob)).sum() / np.log2(prob.size()[0])
print(spectral_entropy(Y, 5), tsf.spectral_entropy(X, 5))
np.isclose(spectral_entropy(Y, 5), tsf.spectral_entropy(X, 5)).all()

tensor(0.9011) 0.9010709860682248


np.True_

In [156]:
import ptwt
def continuous_wavelet_transform(signal, fs, wavelet="mexh", widths=np.arange(1, 10)):
    coefficients, frequencies = ptwt.cwt(signal, widths, wavelet, sampling_period=1 / fs)
    return coefficients, frequencies

def wavelet_entropy(signal, fs, wavelet="mexh", max_width=10):
    if torch.sum(signal) == 0:
        return 0.0
    max_width = int(max_width)
    widths = torch.arange(1, max_width)
    coeffs, _ = continuous_wavelet_transform(signal=signal, fs=fs, wavelet=wavelet, widths=widths)
    energy_scale = torch.sum(torch.abs(coeffs), axis=1)
    t_energy = torch.sum(energy_scale)
    prob = energy_scale / t_energy
    w_entropy = -torch.sum(prob * torch.log(prob))
    return w_entropy
print(wavelet_entropy(Y, 5), tsf.wavelet_entropy(X, 5))
np.isclose(wavelet_entropy(Y, 5), tsf.wavelet_entropy(X, 5)).all()

tensor(2.1943, dtype=torch.float64) 2.194275773109381


np.True_

In [160]:
def wavelet_abs_mean(signal, fs, wavelet="mexh", max_width=10):
    max_width = int(max_width)
    widths = torch.arange(1, max_width)
    coeffs, frequencies = continuous_wavelet_transform(signal=signal, fs=fs, wavelet=wavelet, widths=widths)
    f_keys = np.round(frequencies, 2).astype(str)
    return {"names": [f + "Hz" for f in f_keys], "values": np.abs(torch.mean(coeffs, axis=1))}
print(wavelet_abs_mean(Y, 5))
print(tsf.wavelet_abs_mean(X, 5))

{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': tensor([0.0014, 0.0070, 0.0149, 0.0281, 0.0416, 0.0536, 0.0606, 0.0614, 0.0555],
       dtype=torch.float64)}
{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': array([0.00143808, 0.0070053 , 0.01492294, 0.02811977, 0.04157084,
       0.05357599, 0.06063112, 0.06140213, 0.05549297])}


/tmp/ipykernel_8324/210573967.py:6: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  return {"names": [f + "Hz" for f in f_keys], "values": np.abs(torch.mean(coeffs, axis=1))}


In [161]:
def wavelet_std(signal, fs, wavelet="mexh", max_width=10):
    max_width = int(max_width)
    widths = torch.arange(1, max_width)
    coeffs, frequencies = continuous_wavelet_transform(signal=signal, fs=fs, wavelet=wavelet, widths=widths)
    f_keys = np.round(frequencies, 2).astype(str)
    return {"names": [f + "Hz" for f in f_keys], "values": torch.std(coeffs, axis=1)}
print(wavelet_std(Y, 5))
print(tsf.wavelet_std(X, 5))

{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': tensor([1.0915, 0.9627, 1.0173, 1.0528, 1.0864, 1.1374, 1.1539, 1.1118, 1.0266],
       dtype=torch.float64)}
{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': array([1.08604507, 0.95790926, 1.012239  , 1.04754014, 1.08095311,
       1.13165147, 1.14812133, 1.10618824, 1.02148187])}


In [163]:
def wavelet_var(signal, fs, wavelet="mexh", max_width=10):
    max_width = int(max_width)
    widths = torch.arange(1, max_width)
    coeffs, frequencies = continuous_wavelet_transform(signal=signal, fs=fs, wavelet=wavelet, widths=widths)
    f_keys = np.round(frequencies, 2).astype(str)
    return {"names": [f + "Hz" for f in f_keys], "values": torch.var(coeffs, axis=1)}
print(wavelet_var(Y, 5))
print(tsf.wavelet_var(X, 5))

{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': tensor([1.1914, 0.9269, 1.0350, 1.1084, 1.1803, 1.2936, 1.3315, 1.2360, 1.0540],
       dtype=torch.float64)}
{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': array([1.1794939 , 0.91759014, 1.02462779, 1.09734035, 1.16845962,
       1.28063505, 1.3181826 , 1.22365243, 1.0434252 ])}


In [164]:
def wavelet_energy(signal, fs, wavelet="mexh", max_width=10):
    max_width = int(max_width)
    widths = torch.arange(1, max_width)
    coeffs, frequencies = continuous_wavelet_transform(signal=signal, fs=fs, wavelet=wavelet, widths=widths)
    f_keys = np.round(frequencies, 2).astype(str)
    return {"names": [f + "Hz" for f in f_keys], "values": torch.sqrt(torch.sum(coeffs**2, axis=1) / coeffs.shape[1])}
print(wavelet_energy(Y, 5))
print(tsf.wavelet_energy(X, 5))

{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': tensor([1.0860, 0.9579, 1.0123, 1.0479, 1.0818, 1.1329, 1.1497, 1.1079, 1.0230],
       dtype=torch.float64)}
{'names': ['1.25Hz', '0.62Hz', '0.42Hz', '0.31Hz', '0.25Hz', '0.21Hz', '0.18Hz', '0.16Hz', '0.14Hz'], 'values': array([1.08604602, 0.95793487, 1.01234899, 1.04791749, 1.08175217,
       1.13291899, 1.14972115, 1.10789108, 1.02298811])}


In [165]:
def petrosian_fractal_dimension(signal):
    n = len(signal)
    diff_signal = torch.diff(torch.sign(torch.diff(signal)))
    num_sign_changes = torch.sum(diff_signal != 0)
    pfd = np.log10(n) / (np.log10(n) + np.log10(n / (n + 0.4 * num_sign_changes)))
    return pfd
print(petrosian_fractal_dimension(Y), tsf.petrosian_fractal_dimension(X))
np.isclose(petrosian_fractal_dimension(Y), tsf.petrosian_fractal_dimension(X)).all()

tensor(1.0528) 1.0528369071450172


/tmp/ipykernel_8324/2060534809.py:5: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  pfd = np.log10(n) / (np.log10(n) + np.log10(n / (n + 0.4 * num_sign_changes)))


np.True_